### Pydantic & Dataclasses
+ Pydantic
  - pydantic.dataclasses
+ Dataclasses
      - init, repr, dunder method


#### Tips
+ how to allow extra kwargs in pydantic
+ how to allow extra kwargs in dataclasses


In [1]:
import pydantic
pydantic.__version__

'2.5.3'

In [20]:
from pydantic import BaseModel,ConfigDict

In [21]:
class Books(BaseModel):
    name: str
    price: float

In [22]:
# Create an instance or object
bk = Books(name="Book of Jesse",price=45)

In [23]:
bk

Books(name='Book of Jesse', price=45.0)

In [24]:
bk.name

'Book of Jesse'

In [25]:
bk.price

45.0

In [26]:
# dump as dict
bk.model_dump()

{'name': 'Book of Jesse', 'price': 45.0}

In [28]:
bk = Books(name="Book of Jesse",price=45,author="Jesse")
bk

Books(name='Book of Jesse', price=45.0)

In [29]:
bk.author

AttributeError: 'Books' object has no attribute 'author'

In [ ]:
# Pure python
class BooksPy:
    def __init__(self,name,price,**kwargs):
        self.name = name
        self.price = price
        

In [30]:
class BooksExtra(BaseModel):
    name: str
    price: float
    model_config = ConfigDict(extra="allow")

In [31]:
bk = BooksExtra(name="Book of Jesse",price=45,author="Jesse")
bk

BooksExtra(name='Book of Jesse', price=45.0, author='Jesse')

In [32]:
bk.author

'Jesse'

In [33]:
params = {'name': 'Book of Paul', 'price': 55.0,'author': 'Paul', 'year': 1965}

In [34]:
Books(**params)

Books(name='Book of Paul', price=55.0)

In [35]:
# allow extra
BooksExtra(**params)

BooksExtra(name='Book of Paul', price=55.0, author='Paul', year=1965)

### How to support extra kwargs in Dataclass

In [36]:
from dataclasses import dataclass, field

In [38]:
@dataclass
class BookDC:
    name: str
    price: float
    
   
    

In [40]:
bk2 = BookDC(name='Book of Jesse', price=45.0)

In [41]:
bk2

BookDC(name='Book of Jesse', price=45.0)

In [42]:
bk2.name

'Book of Jesse'

In [43]:
bk2.price

45.0

In [44]:
# Convert to dict
from dataclasses import asdict
asdict(bk2)

{'name': 'Book of Jesse', 'price': 45.0}

In [47]:
# allow extra
print(params)
BookDC(**params)

{'name': 'Book of Paul', 'price': 55.0, 'author': 'Paul', 'year': 1965}


TypeError: BookDC.__init__() got an unexpected keyword argument 'author'

In [48]:
from inspect import signature

def add_from_kwargs(cls):
    def from_kwargs(cls, **kwargs):
        # fetch the constructor's signature
        cls_fields = {field for field in signature(cls).parameters}

        # split the kwargs into native ones and new ones
        native_args, new_args = {}, {}
        for name, val in kwargs.items():
            if name in cls_fields:
                native_args[name] = val
            else:
                new_args[name] = val

        # use the native ones to create the class ...
        ret = cls(**native_args)

        # ... and add the new ones by hand
        for new_name, new_val in new_args.items():
            setattr(ret, new_name, new_val)
        return ret
    cls.from_kwargs = classmethod(from_kwargs)
    return cls
        

In [49]:
@add_from_kwargs
@dataclass
class BookDC:
    name: str
    price: float
    

In [50]:
bk2 = BookDC(name='Book of Jesse', price=45.0)

In [51]:
bk2

BookDC(name='Book of Jesse', price=45.0)

In [52]:
bk2 = BookDC(name='Book of Jesse', price=45.0,author="Jesse")

TypeError: BookDC.__init__() got an unexpected keyword argument 'author'

In [53]:
params = {'name': 'Book of Paul', 'price': 55.0, 'author': 'Paul', 'year': 1965}
bk3 = BookDC.from_kwargs(**params)

In [54]:
bk3

BookDC(name='Book of Paul', price=55.0)

In [55]:
bk3.author

'Paul'

In [56]:
bk3.year

1965

In [59]:
bk4 = BookDC(*params)

TypeError: BookDC.__init__() takes 3 positional arguments but 5 were given

### Thanks 
Jesus Saves 

#### How to Use Nested Data in Pydantic and Dataclasses


In [60]:
from pydantic import BaseModel

class Author(BaseModel):
    author_name: str
    age: int


class Books(BaseModel):
    name: str
    price: float
    author: Author

In [61]:
author = Author(author_name="Paul",age=65)
bk = Books(name="Romans",price=40,author=author)

In [62]:
bk

Books(name='Romans', price=40.0, author=Author(author_name='Paul', age=65))

In [63]:
bk.model_dump()

{'name': 'Romans', 'price': 40.0, 'author': {'author_name': 'Paul', 'age': 65}}

In [65]:
bk.model_json_schema()

{'$defs': {'Author': {'properties': {'author_name': {'title': 'Author Name',
     'type': 'string'},
    'age': {'title': 'Age', 'type': 'integer'}},
   'required': ['author_name', 'age'],
   'title': 'Author',
   'type': 'object'}},
 'properties': {'name': {'title': 'Name', 'type': 'string'},
  'price': {'title': 'Price', 'type': 'number'},
  'author': {'$ref': '#/$defs/Author'}},
 'required': ['name', 'price', 'author'],
 'title': 'Books',
 'type': 'object'}

In [66]:
# Traverse 
bk.author

Author(author_name='Paul', age=65)

In [67]:
bk.author.age

65

In [70]:
# Create another instance from json/dict
data = {'name': 'Harry Porter', 'price': 40.0, 'author': {'author_name': 'Paul', 'age': 65}}
bk2 = Books(**data)

In [71]:
bk2

Books(name='Harry Porter', price=40.0, author=Author(author_name='Paul', age=65))

In [72]:
bk2.author.author_name

'Paul'

### Using Dataclasses For Nested Data

In [73]:
from dataclasses import dataclass, asdict

In [74]:
@dataclass
class AuthorDC:
    author_name: str
    age: int


@dataclass
class BookDC:
    name: str
    price: float
    author: Author
    

In [75]:
author_dc = AuthorDC(author_name="Enoch",age=365)
bkdc1 = BookDC(name="Book of Enoch",price=40,author=author_dc)

In [76]:
bkdc1

BookDC(name='Book of Enoch', price=40, author=AuthorDC(author_name='Enoch', age=365))

In [77]:
bkdc1.name

'Book of Enoch'

In [78]:
bkdc1.author

AuthorDC(author_name='Enoch', age=365)

In [79]:
bkdc1.author.age

365

In [80]:
data

{'name': 'Harry Porter',
 'price': 40.0,
 'author': {'author_name': 'Paul', 'age': 65}}

In [82]:
# create an instance from the nested data
bk3 = BookDC(**data)

In [83]:
bk3

BookDC(name='Harry Porter', price=40.0, author={'author_name': 'Paul', 'age': 65})

In [86]:
bk3.author

{'author_name': 'Paul', 'age': 65}

In [87]:
bk3.author.age

AttributeError: 'dict' object has no attribute 'age'

In [96]:
# Solution is the post init
@dataclass
class BookDCwithPostInit:
    name: str
    price: float
    author: Author

    def __post_init__(self):
        if isinstance(self.author,dict):
            self.author = Author(**self.author)
            

In [89]:
data

{'name': 'Harry Porter',
 'price': 40.0,
 'author': {'author_name': 'Paul', 'age': 65}}

In [97]:
# unpack the data and initialize
# create an instance from the nested data
bk4 = BookDCwithPostInit(**data)

In [99]:
bk4

BookDCwithPostInit(name='Harry Porter', price=40.0, author=Author(author_name='Paul', age=65))

In [100]:
bk4.name

'Harry Porter'

In [101]:
bk4.author

Author(author_name='Paul', age=65)

In [102]:
bk4.author.age

65

In [1]:
### Thanks For Your Time
#### Jesus Saves @JCharisTech
#### Jesse E.Agbe (JCharis)